<a href="https://colab.research.google.com/github/onlyabhilash/spark-nlp-ClinicalResearch/blob/main/4_2_Clinical_Deidentification_in_Spanish/4_2_Clinical_Deidentification_in_Spanish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/4.2.Clinical_Deidentification_in_Spanish.ipynb)

# Clinical Deidentification in Spanish

**Protected Health Information**:

Individual’s past, present, or future physical or mental health or condition
provision of health care to the individual
past, present, or future payment for the health care
Protected health information includes many common identifiers (e.g., name, address, birth date, Social Security Number) when they can be associated with the health information.

In [1]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_4790.json to spark_nlp_for_healthcare_spark_ocr_4790.json


In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

     |████████████████████████████████| 212.4 MB 72 kB/s 
     |████████████████████████████████| 198 kB 47.4 MB/s 
     |████████████████████████████████| 188 kB 4.9 MB/s 
     |████████████████████████████████| 142 kB 5.5 MB/s 


In [3]:
import sys
import os
import json
import pandas as pd
import string
import numpy as np

import sparknlp
import sparknlp_jsl

from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import ResourceDownloader
from sparknlp.util import *
from sparknlp_jsl.annotator import *

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(SECRET, params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.2
Spark NLP_JSL Version : 3.5.0


# 1. Spanish NER Deidentification Models
We have eight different models you can use:
* `ner_deid_generic`, detects 7 entities, uses SciWiki 300d embeddings.
* `ner_deid_generic_roberta`, same as previous, but uses Roberta Clinical Embeddings.
* `ner_deid_generic_augmented`, detects 8 entities (now includes 'SEX' entity), uses SciWiki 300d embeddings and has been trained with more data
* `ner_deid_generic_roberta_augmented`, same as previous, but uses Roberta Clinical Embeddings.
* `ner_deid_subentity`, detects 13 entities, uses SciWiki 300d embeddings.
* `ner_deid_subentity_roberta`, same as previous, but uses Roberta Clinical Embeddings.
* `ner_deid_subentity_augmented`, detects 17 entities, uses SciWiki 300d embeddings and has been trained with more data.
* `ner_deid_subentity_roberta_augmented`, same as previous, but uses Roberta Clinical Embeddings.

Since `augmented` models improve their results compared to the non augmented ones, we are going to show case them in this notebook

|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [ner_deid_subentity]()| 2| [ner_deid_generic ]()|
| 3| [ner_deid_subentity_augmented]()| 4| [ner_deid_generic_augmented]()|
| 5| [ner_deid_subentity_roberta]()| 6| [ner_deid_generic_roberta]()|
| 7| [ner_deid_subentity_roberta_augmented ]()| 8| [ner_deid_generic_roberta_augmented]()|

### Creating pipeline for Sciwiki 300d-based augmented model

In [4]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx") \
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_sciwiki_300d","es","clinical/models")\
    .setInputCols(["document","token"])\
	  .setOutputCol("embeddings")

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
embeddings_sciwiki_300d download started this may take some time.
Approximate size to download 253.3 MB
[OK!]


## 1.1. NER Deid Generic (Augmented)

**`ner_deid_generic_augmented`** extracts:
- Name
- Profession
- Age
- Date
- Contact (Telephone numbers, FAX numbers, Email addresses)
- Location (Address, City, Postal code, Hospital Name, Employment information)
- Id (Social Security numbers, Medical record numbers, Internet protocol addresses)
- Sex



In [5]:
ner_generic = MedicalNerModel.pretrained("ner_deid_generic_augmented", "es", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_generic")

ner_converter_generic = NerConverter()\
    .setInputCols(["sentence","token","ner_deid_generic"])\
    .setOutputCol("ner_chunk_generic")

ner_deid_generic_augmented download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [6]:
ner_generic.getClasses()

['O',
 'I-LOCATION',
 'B-ORGANIZATION',
 'I-CONTACT',
 'I-PROFESSION',
 'I-NAME',
 'I-DATE',
 'B-ID',
 'B-PROFESSION',
 'B-CONTACT',
 'I-ID',
 'B-NAME',
 'B-DATE',
 'B-LOCATION',
 'B-SEX',
 'I-ORGANIZATION',
 'B-AGE',
 'I-SEX']

## 1.2. NER Deid Subentity

**`ner_deid_subentity`** extracts:

- Patient
- Doctor
- Hospital
- Date
- Organization
- City
- Street
- User Name
- Profession
- Phone
- Country
- Age
- Sex
- Email
- ZIP
- ID
- Medical Record

In [7]:
ner_subentity = MedicalNerModel.pretrained("ner_deid_subentity_augmented", "es", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_subentity")

ner_converter_subentity = NerConverter()\
    .setInputCols(["sentence", "token", "ner_deid_subentity"])\
    .setOutputCol("ner_chunk_subentity")

ner_deid_subentity_augmented download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [8]:
ner_subentity.getClasses()

['O',
 'B-MEDICALRECORD',
 'B-ORGANIZATION',
 'I-PROFESSION',
 'B-DOCTOR',
 'B-USERNAME',
 'B-PROFESSION',
 'I-ID',
 'B-CITY',
 'B-DATE',
 'B-PATIENT',
 'B-SEX',
 'I-SEX',
 'I-DOCTOR',
 'I-CITY',
 'I-DATE',
 'B-COUNTRY',
 'B-ID',
 'B-ZIP',
 'I-STREET',
 'I-PATIENT',
 'B-PHONE',
 'I-PHONE',
 'B-HOSPITAL',
 'B-EMAIL',
 'B-STREET',
 'I-ORGANIZATION',
 'B-AGE',
 'I-HOSPITAL',
 'I-COUNTRY']

## 1.4. Pipeline

In [9]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_generic,
      ner_converter_generic,
      ner_subentity,
      ner_converter_subentity,
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [10]:
text = "Antonio Miguel Martínez, un varón de 35 años de edad, de profesión auxiliar de enfermería y nacido en Cadiz, España. Aún no estaba vacunado, se infectó con Covid-19 el dia 14/03/2022 y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la Clinica San Carlos."

text_df = spark.createDataFrame([[text]]).toDF("text")

result = model.transform(text_df)


### Results for `ner_subentity`

In [11]:
result.select(F.explode(F.arrays_zip('ner_chunk_subentity.result', 'ner_chunk_subentity.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+----------+
|chunk                  |ner_label |
+-----------------------+----------+
|Antonio Miguel Martínez|PATIENT   |
|un varón               |SEX       |
|35                     |AGE       |
|auxiliar de enfermería |PROFESSION|
|Cadiz                  |CITY      |
|España                 |COUNTRY   |
|14/03/2022             |DATE      |
|Clinica San Carlos     |HOSPITAL  |
+-----------------------+----------+



### Results for `ner_generic`

In [12]:
result.select(F.explode(F.arrays_zip('ner_chunk_generic.result', 'ner_chunk_generic.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+----------+
|chunk                  |ner_label |
+-----------------------+----------+
|Antonio Miguel Martínez|NAME      |
|un varón               |SEX       |
|35                     |AGE       |
|auxiliar de enfermería |PROFESSION|
|Cadiz                  |LOCATION  |
|España                 |LOCATION  |
|14/03/2022             |DATE      |
|Clinica San Carlos     |LOCATION  |
+-----------------------+----------+



## DeIdentification

### Obfuscation mode

In [13]:
# Downloading faker entity list.
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/obfuscate_es.txt

In [14]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate_es.txt')\
    .setObfuscateRefSource("file")\

In [15]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [16]:
deid_lp = LightPipeline(model)

In [17]:
text = "Antonio Miguel Martínez, un varón de 35 años de edad, de profesión auxiliar de enfermería y nacido en Cadiz, España. Aún no estaba vacunado, se infectó con Covid-19 el dia 14/03/2022 y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la Clinica San Carlos."

In [18]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

<PATIENT>, <SEX> de <AGE> años de edad, de profesión <PROFESSION> y nacido en <CITY>, <COUNTRY>.
Aún no estaba vacunado, se infectó con Covid-19 el dia <DATE> y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la <HOSPITAL>.


[*********************], [******] de ** años de edad, de profesión [********************] y nacido en [***], [****].
Aún no estaba vacunado, se infectó con Covid-19 el dia [********] y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la [****************].


****, **** de **** años de edad, de profesión **** y nacido en ****, ****.
Aún no estaba vacunado, se infectó con Covid-19 el dia **** y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la ****.


Antonio González Cuevas, M. de 26 años de edad, de profesión Docente y nacido en Zaragoza, Portugal.
Aún no estaba vacunado, se infectó con Covid-19 el dia 24/03/2022 y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la Hospital General

In [19]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["masked_with_entity"], 
                           result["masked_with_chars"],
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                  columns= ["Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"<PATIENT>, <SEX> de <AGE> años de edad, de profesión <PROFESSION> y nacido en <CITY>, <COUNTRY>.","[*********************], [******] de ** años de edad, de profesión [********************] y naci...","****, **** de **** años de edad, de profesión **** y nacido en ****, ****.","Antonio González Cuevas, M. de 26 años de edad, de profesión Docente y nacido en Zaragoza, Portu..."
1,"Aún no estaba vacunado, se infectó con Covid-19 el dia <DATE> y tuvo que ir al Hospital. Fue tra...","Aún no estaba vacunado, se infectó con Covid-19 el dia [********] y tuvo que ir al Hospital. Fue...","Aún no estaba vacunado, se infectó con Covid-19 el dia **** y tuvo que ir al Hospital. Fue trata...","Aún no estaba vacunado, se infectó con Covid-19 el dia 24/03/2022 y tuvo que ir al Hospital. Fue..."


### Faker Mode

In [20]:
deid_masked_entity = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_entity")\
    .setMode("mask")\
    .setMaskingPolicy("entity_labels")

deid_masked_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_with_chars")\
    .setMode("mask")\
    .setMaskingPolicy("same_length_chars")

deid_masked_fixed_char = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"])\
    .setOutputCol("masked_fixed_length_chars")\
    .setMode("mask")\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)

deid_obfuscated = DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk_subentity"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate")\
    .setLanguage('es')\
    .setObfuscateDate(True)\
    .setObfuscateRefSource('faker')

In [21]:
nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentencerDL,
      tokenizer,
      word_embeddings,
      ner_subentity,
      ner_converter_subentity,
      deid_masked_entity,
      deid_masked_char,
      deid_masked_fixed_char,
      deid_obfuscated
      ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

In [22]:
deid_lp = LightPipeline(model)

In [23]:
text = "Antonio Miguel Martínez, un varón de 35 años de edad, de profesión auxiliar de enfermería y nacido en Cadiz, España. Aún no estaba vacunado, se infectó con Covid-19 el dia 14/03/2022 y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la Clinica San Carlos."

In [24]:
result = deid_lp.annotate(text)

print("\n".join(result['masked_with_entity']))
print("\n")
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

<PATIENT>, <SEX> de <AGE> años de edad, de profesión <PROFESSION> y nacido en <CITY>, <COUNTRY>.
Aún no estaba vacunado, se infectó con Covid-19 el dia <DATE> y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la <HOSPITAL>.


[*********************], [******] de ** años de edad, de profesión [********************] y nacido en [***], [****].
Aún no estaba vacunado, se infectó con Covid-19 el dia [********] y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la [****************].


****, **** de **** años de edad, de profesión **** y nacido en ****, ****.
Aún no estaba vacunado, se infectó con Covid-19 el dia **** y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la ****.


Aser Estebanez, H de 73 años de edad, de profesión Designer, jewellery y nacido en Guixers, Mauritania.
Aún no estaba vacunado, se infectó con Covid-19 el dia 23/04/2022 y tuvo que ir al Hospital. Fue tratado con anticuerpos monoclonales en la Hospital Infa

In [25]:
pd.set_option("display.max_colwidth", 100)

df = pd.DataFrame(list(zip(result["masked_with_entity"], 
                           result["masked_with_chars"],
                           result["masked_fixed_length_chars"], 
                           result["obfuscated"])),
                  columns= ["Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"<PATIENT>, <SEX> de <AGE> años de edad, de profesión <PROFESSION> y nacido en <CITY>, <COUNTRY>.","[*********************], [******] de ** años de edad, de profesión [********************] y naci...","****, **** de **** años de edad, de profesión **** y nacido en ****, ****.","Aser Estebanez, H de 73 años de edad, de profesión Designer, jewellery y nacido en Guixers, Maur..."
1,"Aún no estaba vacunado, se infectó con Covid-19 el dia <DATE> y tuvo que ir al Hospital. Fue tra...","Aún no estaba vacunado, se infectó con Covid-19 el dia [********] y tuvo que ir al Hospital. Fue...","Aún no estaba vacunado, se infectó con Covid-19 el dia **** y tuvo que ir al Hospital. Fue trata...","Aún no estaba vacunado, se infectó con Covid-19 el dia 23/04/2022 y tuvo que ir al Hospital. Fue..."


# 2. Pretrained Spanish Deidentification Pipeline

- We developed a clinical deidentification pretrained pipeline that can be used to deidentify PHI information from German medical texts. The PHI information will be masked and obfuscated in the resulting text. 
- The pipeline can mask and obfuscate:
    - Patient
    - Doctor
    - Hospital
    - Date
    - Organization
    - City
    - Street
    - Country
    - User name
    - Profession
    - Phone
    - Age
    - Contact
    - ID
    - Phone
    - ZIP
    - Account
    - SSN
    - Driver's License Number
    - Plate Number
    - Sex

|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [clinical_deidentification_augmented]()| 2| [clinical_deidentification]()|

In [26]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification_augmented", "es", "clinical/models")

clinical_deidentification_augmented download started this may take some time.
Approx size to download 268.2 MB
[OK!]


In [27]:
text = """Datos del paciente.
Nombre:  Ernesto.
Apellidos: Rivera Bueno.
NHC: 368503.
NASS: 26 63514095.
Domicilio:  Calle Miguel Benitez 90.
Localidad/ Provincia: Madrid.
CP: 28016.
Datos asistenciales.
Fecha de nacimiento: 03/03/1946.
País: España.
Edad: 70 años Sexo: H.
Fecha de Ingreso: 12/12/2016.
Médico:  Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Informe clínico del paciente: Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, con abdomen y genitales normales; tacto rectal compatible con adenoma de próstata grado I/IV.
En la analítica de orina destaca la existencia de 4 hematíes/ campo y 0-5 leucocitos/campo; resto de sedimento normal.
Hemograma normal; en la bioquímica destaca una glucemia de 169 mg/dl y triglicéridos de 456 mg/dl; función hepática y renal normal. PSA de 1.16 ng/ml.
Las citologías de orina son repetidamente sospechosas de malignidad.
En la placa simple de abdomen se valoran cambios degenerativos en columna lumbar y calcificaciones vasculares en ambos hipocondrios y en pelvis.
La ecografía urológica pone de manifiesto la existencia de quistes corticales simples en riñón derecho, vejiga sin alteraciones con buena capacidad y próstata con un peso de 30 g.
En la UIV se observa normofuncionalismo renal bilateral, calcificaciones sobre silueta renal derecha y uréteres arrosariados con imágenes de adición en el tercio superior de ambos uréteres, en relación a pseudodiverticulosis ureteral. El cistograma demuestra una vejiga con buena capacidad, pero paredes trabeculadas en relación a vejiga de esfuerzo. La TC abdominal es normal.
La cistoscopia descubre la existencia de pequeñas tumoraciones vesicales, realizándose resección transuretral con el resultado anatomopatológico de carcinoma urotelial superficial de vejiga.
Remitido por: Ignacio Navarro Cuéllar c/ del Abedul 5-7, 2º dcha 28036 Madrid, España E-mail: nnavcu@hotmail.com.
"""

result = deid_pipeline.annotate(text)
print("\n".join(result['masked_with_chars']))
print("\n")
print("\n".join(result['masked']))
print("\n")
print("\n".join(result['masked_fixed_length_chars']))
print("\n")
print("\n".join(result['obfuscated']))

Datos [**********].
Nombre:  [*****].
Apellidos: [**********].
NHC: [****].
NASS: [*********].
Domicilio:  [*********************].
Localidad/ Provincia: [****].
CP: [***].
Datos asistenciales.
Fecha de nacimiento: [********].
País: [****].
Edad: ** años Sexo: *.
Fecha de Ingreso: [********].
Médico:  [*********************] NºCol: [*********].
Informe clínico [**********]: [******] ** ** años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo;
Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
En la exploración física presenta un buen estado general, 

# Other NER versions: Using Roberta Clinical Embeddings based NER
You can use also Roberta Clinical Embeddings and `_roberta` , instead of Sciwi for NER models (not for Pretrained Pipeline, that comes only with `Sciwi`). This is an example of how to use the 

In [28]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

roberta_embeddings = RoBertaEmbeddings.pretrained("roberta_base_biomedical", "es")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained("ner_deid_subentity_roberta_augmented", "es", "clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
      documentAssembler,
      sentenceDetector,
      tokenizer,
      roberta_embeddings,
      clinical_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
roberta_base_biomedical download started this may take some time.
Approximate size to download 288 MB
[OK!]
ner_deid_subentity_roberta_augmented download started this may take some time.
Approximate size to download 15.6 MB
[OK!]


In [29]:
text = ['''
Antonio Miguel Martínez, varón de de 35 años de edad, de profesión auxiliar de enfermería y nacido en Cadiz, España. Aún no estaba vacunado, se infectó con Covid-19 el dia 14 de Marzo y tuvo que ir al Hospital
Fue tratado con anticuerpos monoclonales en la Clinica San Carlos.
''']

In [30]:
result = model.transform(spark.createDataFrame([text]).toDF("text"))

In [31]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+----------+
|chunk                  |ner_label |
+-----------------------+----------+
|Antonio Miguel Martínez|PATIENT   |
|varón                  |SEX       |
|35                     |AGE       |
|auxiliar de enfermería |PROFESSION|
|Cadiz                  |CITY      |
|España                 |COUNTRY   |
|14 de Marzo            |DATE      |
|Clinica San Carlos     |HOSPITAL  |
+-----------------------+----------+



# About non-augmented models
You can use any of the previous models without the `_augmented` suffix. However, those models were trained with less data and have less entities, so we highly recommend to use the `augmented` versions.